# Baseline Experiment - Pre-optimized Solution

This notebook loads the best pre-optimized solution from snapshots and verifies its score.

In [ ]:
import pandas as pd
import numpy as np
import math
from numba import njit
import os

# Tree geometry
TX = np.array([0, 0.125, 0.0625, 0.2, 0.1, 0.35, 0.075, 0.075, -0.075, -0.075, -0.35, -0.1, -0.2, -0.0625, -0.125])
TY = np.array([0.8, 0.5, 0.5, 0.25, 0.25, 0, 0, -0.2, -0.2, 0, 0, 0.25, 0.25, 0.5, 0.5])

print(f"Tree has {len(TX)} vertices")
print(f"Tree width: {max(TX) - min(TX):.3f}")
print(f"Tree height: {max(TY) - min(TY):.3f}")

In [ ]:
@njit
def score_group(xs, ys, degs, tx, ty):
    """Calculate score for a single N-tree configuration"""
    n = xs.size
    V = tx.size
    mnx = mny = 1e300
    mxx = mxy = -1e300
    for i in range(n):
        r = degs[i] * math.pi / 180.0
        c, s = math.cos(r), math.sin(r)
        for j in range(V):
            X = c * tx[j] - s * ty[j] + xs[i]
            Y = s * tx[j] + c * ty[j] + ys[i]
            mnx, mxx = min(mnx, X), max(mxx, X)
            mny, mxy = min(mny, Y), max(mxy, Y)
    side = max(mxx - mnx, mxy - mny)
    return side * side / n

def calculate_total_score(df):
    """Calculate total score for a submission dataframe"""
    total_score = 0.0
    scores_by_n = {}
    
    for n in range(1, 201):
        # Get rows for this N
        mask = df['id'].str.startswith(f'{n:03d}_')
        group = df[mask]
        
        if len(group) != n:
            print(f"Warning: N={n} has {len(group)} trees, expected {n}")
            continue
        
        # Parse coordinates (remove 's' prefix)
        xs = group['x'].str[1:].astype(float).values
        ys = group['y'].str[1:].astype(float).values
        degs = group['deg'].str[1:].astype(float).values
        
        # Calculate score
        score = score_group(xs, ys, degs, TX, TY)
        scores_by_n[n] = score
        total_score += score
    
    return total_score, scores_by_n

print("Scoring functions defined")

In [ ]:
# Load the pre-optimized submission
submission_path = '/home/nonroot/snapshots/santa-2025/21328309254/code/submission.csv'
df = pd.read_csv(submission_path)
print(f"Loaded submission with {len(df)} rows")
print(df.head())

In [ ]:
# Calculate total score
total_score, scores_by_n = calculate_total_score(df)
print(f"\nTotal Score: {total_score:.6f}")
print(f"Expected: 70.647327")
print(f"Difference: {abs(total_score - 70.647327):.6f}")

In [ ]:
# Analyze score contribution by N
print("\nTop 20 N values by score contribution:")
sorted_scores = sorted(scores_by_n.items(), key=lambda x: x[1], reverse=True)
for n, score in sorted_scores[:20]:
    print(f"  N={n:3d}: {score:.6f}")

In [ ]:
# Calculate packing efficiency for each N
print("\nPacking efficiency analysis:")
tree_area = 0.3325  # Approximate area of tree polygon

for n in [1, 2, 5, 10, 20, 50, 100, 200]:
    score = scores_by_n[n]
    side = math.sqrt(score * n)
    box_area = side * side
    total_tree_area = n * tree_area
    efficiency = total_tree_area / box_area * 100
    print(f"  N={n:3d}: side={side:.4f}, efficiency={efficiency:.1f}%")

In [ ]:
# Copy to submission folder
import shutil
os.makedirs('/home/submission', exist_ok=True)
shutil.copy(submission_path, '/home/submission/submission.csv')
print("Submission copied to /home/submission/submission.csv")

# Verify the copy
df_verify = pd.read_csv('/home/submission/submission.csv')
print(f"Verified: {len(df_verify)} rows")

In [ ]:
# Save metrics
import json
metrics = {
    'cv_score': total_score,
    'scores_by_n': {str(k): v for k, v in scores_by_n.items()}
}

with open('/home/code/experiments/001_baseline/metrics.json', 'w') as f:
    json.dump(metrics, f, indent=2)

print(f"Metrics saved. CV Score: {total_score:.6f}")